<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1b/EBay_logo.svg/2880px-EBay_logo.svg.png" alt="eBay Logo" title="eBay Logo" width=300px/>
# eBay Car Sales Data
#### Author: Frank Pereny
#### Date November, 2020

## Introduction:
### Project Summary:
[eBay Kleinanzeigen](https://www.ebay-kleinanzeigen.de/) (ebay Classifieds), is the German section of eBay.  Used car classifies were collected and analyzed to indentify pricing and branding and other information were analyzed to understand the German eBay auto marketplace.

### Goals:
Cleaning the data to perform the following:

- Analysis of registration year (age) of the vehicles for sale
- Analysis of ad posting dates
- Most popular and least popular brand of vehicles
- Average eBay listing price by brand
- Average odometer reading by brand
- "Value Index" which compare the value retained by brand using both average price and odometer readings


### Source Data:
#### Original Source Data:
The original source data was [scraped](https://en.wikipedia.org/wiki/Web_scraping) and uploaded to [Kaggle](https://www.kaggle.com/) by user [orgesleka](https://www.kaggle.com/orgesleka).  The data no longer exists on Kaggle, but it can be [downloaded here](https://data.world/data-society/used-cars-data).
#### Project Source Data:
The data used in this project is a sub-set of the orignal data.  The following modifications were made to the data by [DATAQUEST](https://www.dataquest.io/) for the purposes of this project.

1) Data set was reduced to 50,000 data points.

2) Since the original data had been cleaned, the data was "dirtied" to more closely resemble raw scraped data.

#### Data Dictionary:

|Label | Meaning|
|----- | -------|
| dateCrawled | When ad was first crawled.
| name| Name of the car. | 
| seller | Whether seller is a dealer or private party.|
| offerType | The type of listing.|
|price| The price on the ad to sell the car |
|abtest | Whether the listing is inlcuded in an A/B test.|
|vehicleType| The type of vehicle for sale.|
|yearOfRegistration| The year in which the car was first registered.|
|gearbox | Manual or automatic transmission.|
|powerPS | The power of the car's engine in DIN [horsepower](https://en.wikipedia.org/wiki/Horsepower) (German pferdestrke "horse strength"). 1 DIN hp is approximately 735.5 watt and should not be confused with SAE horsepower (745.7 watt).|
|model | Car model name|
|kilometer | Odometer reading in kilometers.|
|monthOfRegistration| The month in which the car was first registered.|
|fuelType | What type of fuel the car uses.|
|brand| The brand of the car.|
|notRepairedDamage| If the car has damage that has not been repaired.|
|dateCreated| The date that the eBay listing was created.|
|nrOfPictures| The number of photos in the ad.|
|postalCode| The postal code for the location of the vehicle.|
|lastSeenOnline | When the crawler last saw this ad online.|


#### Data Cleaning
Data was cleaned in the following order.  Based on our cleaning criteria, almost 95% of the original data set was retained for anlaysis.

| Number of Filters Appplied |   Filter    | Data Points | Percent | Data Lost (Cumulutive) |
|----------------------------|------------ | --------    | --------| ------                 |
|             0              | Original Source   | 50,000 | 100%   | 0%    |
|              1             | Car Price >= \$200 | 47,659 | 95.32% | 4.68% |
|              2             | Car Price <= \$350,000 | 47,645 | 95.29% | 4.71% |
|             3              |1900 <= Car Registration Year <= 2007 | 47,157 | 94.31% | 5.69%|



In [1]:
import numpy as np
import pandas as pd

## Analysis
### CSV File Encoding
From the error shown below we know that the project data file is not UTF-8 encoded.

In [2]:
try:
    autos_UTF8 = pd.read_csv('autos.csv')
except:
    print('File is not UTF-8 encoded!')
    
try: 
    autos_Latin1 = pd.read_csv('autos.csv', encoding='Latin-1')
except:
    print('File is not Latin-1 encoded!')
    
try:
    autos_Windows1252 = pd.read_csv('autos.csv', encoding='Windows-1252')
except:
    print('File is not Windows-1252 encoded!')

File is not UTF-8 encoded!


We can inspect the Latin-1 and Windows-1252 encodings to determine which may be be the correct encoding.  Based on the results below it seems that either data set would be acceptable.  For the remainder of the project we will use the Latin-1 encoded data set and assign it to autos.

In [3]:
autos_Latin1.head(3)

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37


In [4]:
autos_Windows1252.head(3)

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37


We can make the following observations about the data set:

- 50,000 data points
- 20 Columns of data
- There are two types of data, in64 and string
    - 5 Columns of in64 type data
    - 15 columns of string type data
    - There is no null data

In [5]:
autos = autos_Latin1
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

### Column Names
#### Naming
Column names were changed to match [snakecase](https://en.wikipedia.org/wiki/Snake_case) and the following:

- Lower case
- Words separated by underscore
- Removed unnessessary words:
    - yearOfRegistration change to registration_year
    - monthOfRegistration changed to registration_month
- Improved readability:
    - notRepairedDamage changed to unrepaired_damage
    - dateCreated changed to ad_created
    - nrOfPictures changed to num_of_pictures
    - powerPS changed to hp


In [6]:
new_columns = [
    'date_crawled',
    'name',
    'seller',
    'offer_type',
    'price',
    'ab_test',
    'vehicle_type',
    'registration_year',
    'gearbox',
    'hp',
    'model',
    'odometer',
    'registration_month',
    'fuel_type',
    'brand',
    'unrepaired_damage',
    'ad_created',
    'num_of_pictures',
    'postal_code',
    'last_seen'   
]

autos.columns = new_columns
autos.head(1)

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,hp,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54


### Data Cleaning

#### Initial Observations
##### Numeric Data
The following analysis shows some problems with the numerical data that can be corrected.

- ***registration_year***
    - Minimum registration year is 1000.
    - Maximum registration year is 9999.
- ***hp***
    - Minimum hp is 0, which must be an error.
- ***registration_month***
    - Minimum registration month is 0.
- ***num_of_pictures***
    - This data is 0 for all 50,000 rows so has no value.
- ***postal_code***
    - [German Postal Codes](https://en.wikipedia.org/wiki/List_of_postal_codes_in_Germany) have been 5 digits since 1993, before the start of eBay.
    - Some postal codes are less than 5 digits in length.
    - Some postal codes are more than 5 digits in length.

In [7]:
autos.describe()

,registration_year,hp,registration_month,num_of_pictures,postal_code
count,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,2005.073280,116.355920,5.723360,0.0,50813.627300
std,105.712813,209.216627,3.711984,0.0,25779.747957
min,1000.000000,0.000000,0.000000,0.0,1067.000000
25%,1999.000000,70.000000,3.000000,0.0,30451.000000
50%,2003.000000,105.000000,6.000000,0.0,49577.000000
75%,2008.000000,150.000000,9.000000,0.0,71540.000000
max,9999.000000,17700.000000,12.000000,0.0,99998.000000


##### Non-Numeric Data
- ***date_crawled***
    - Nothing unusual.
- ***name***
    - Nothing unusual.
- ***seller***
    - 49,999 of 50,000 are 'privat' (English - private)
    - Column data has no value.
- ***offer_type***
    - 49,999 of 50,000 are 'Angebot' (English - offer)
    - Column data has no value.
- ***price***
    - Top price is \\$0 (free) which may be an error
    - Should be numeric
- ***ab_test***
    - Nothing unusual
- ***vehicle_type***
    - 44,905 count (should be 50,000)
- ***gearbox***
    - 47,320 count (should be 50,000)
- ***model***
    - 47,242 count (should be 50,000)
- ***odometer***
    - Should be numeric
    - Should change name of column to include units (km)
- ***fuel_type***
    - 45,518 count (should be 50,000)
    - 7 unique fuel types might be an error
        - 3 unique types (gas, diesel, electric) were expected
        - Perhaps different grades of fuel, for example [gasoline octane](https://en.wikipedia.org/wiki/Octane_rating), are specified
        - Should be further investigated
- ***brand***
    - Nothing unusual
- ***unrepaired_damage***
    - Nothing unusual
- ***date_created***
    - Nothing unusual
- ***last_seen***
    - Nothing unusual


In [8]:
autos.describe(include=np.object)

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,gearbox,model,odometer,fuel_type,brand,unrepaired_damage,ad_created,last_seen
count,50000,50000,50000,50000,50000,50000,44905,47320,47242,50000,45518,50000,40171,50000,50000
unique,48213,38754,2,2,2357,2,8,2,245,13,7,40,2,76,39481
top,2016-03-27 22:55:05,Ford_Fiesta,privat,Angebot,$0,test,limousine,manuell,golf,"150,000km",benzin,volkswagen,nein,2016-04-03 00:00:00,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,36993,4024,32424,30107,10687,35232,1946,8


#### Converting 'price' and 'odometer' to Numeric Data


In [9]:
def string_to_float(string):
    # We only want to keep the digits and the decimal point in the string
    # The following is a string which stores the allowed characters
    allowed_chars = '0123456789.'
    
    # We can create a new string using the allowed_chars to act as a filter
    new_string = ''.join(c for c in string if c in allowed_chars)
    
    # Convert string to float and return the value
    return float(new_string)

autos["price"] = autos["price"].apply(string_to_float)
autos["odometer"] = autos["odometer"].apply(string_to_float)

We can see that price and odometer are float type data.

In [10]:
autos.describe()

,price,registration_year,hp,odometer,registration_month,num_of_pictures,postal_code
count,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,9.840044e+03,2005.073280,116.355920,125732.700000,5.723360,0.0,50813.627300
std,4.811044e+05,105.712813,209.216627,40042.211706,3.711984,0.0,25779.747957
min,0.000000e+00,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1.100000e+03,1999.000000,70.000000,125000.000000,3.000000,0.0,30451.000000
50%,2.950000e+03,2003.000000,105.000000,150000.000000,6.000000,0.0,49577.000000
75%,7.200000e+03,2008.000000,150.000000,150000.000000,9.000000,0.0,71540.000000
max,1.000000e+08,9999.000000,17700.000000,150000.000000,12.000000,0.0,99998.000000


Renaming ***odometer*** to ***odometer_km*** since km was removed from all data.

In [11]:
new_column = {'odometer': 'odometer_km'}
autos.rename(columns=new_column, inplace=True)
autos.head(1)

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,hp,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000.0,control,bus,2004,manuell,158,andere,150000.0,3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54


### Numeric Data Cleaning
Analysis of numeric data:
- ***price***
    - Some prices too low (free)
    - Some prices too high (100,000,000)
- ***registration_year***
    - Minimum registration year is 1000.
    - Maximum registration year is 9999.
- ***hp***
    - Minimum hp is 0
- ***odometer_km***
    - Nothing unusual.
- ***registration_month***
    - Minimum registration month is 0.
- ***num_of_pictures***
    - This data is 0 for all 50,000 rows so has no value.
- ***postal_code***
    - [German Postal Codes](https://en.wikipedia.org/wiki/List_of_postal_codes_in_Germany) have been 5 digits since 1993, before the start of eBay.
    - Some postal codes are less than 5 digits in length.
    - Some postal codes are more than 5 digits in length.

## Price
### Cleaning 'price'
Price has values that seem to high or too low to be possible.  There are no NaN in the data.
#### Checking for NaN


In [12]:
autos['price'].isnull().sum()

0

#### Top 20 Lowest Prices
Based on review of the lowest prices it is clear that many values are much too low to be considered reasonable.  Most of them are zero, but there are many that should be fixed or removed.

In [13]:
autos['price'].value_counts().sort_index().head(20)

0.0     1421
1.0      156
2.0        3
3.0        1
5.0        2
8.0        1
9.0        1
10.0       7
11.0       2
12.0       3
13.0       2
14.0       1
15.0       2
17.0       3
18.0       1
20.0       4
25.0       5
29.0       1
30.0       7
35.0       1
Name: price, dtype: int64

If we remove all cars that are less than $200 from the list, we are left with 47,659 of 50,000 data points.  Since this retains over 95% of the original data, it seems like a reasonable compromise.

In [14]:
autos = autos.loc[autos['price'] >= 200, :]
autos.shape

(47659, 20)

#### Top 20 Highest Prices
The top 20 highest prices reveal some impossibly high values.

In [15]:
autos['price'].value_counts().sort_index(ascending=False).head(20)

99999999.0    1
27322222.0    1
12345678.0    3
11111111.0    2
10000000.0    1
3890000.0     1
1300000.0     1
1234566.0     1
999999.0      2
999990.0      1
350000.0      1
345000.0      1
299000.0      1
295000.0      1
265000.0      1
259000.0      1
250000.0      1
220000.0      1
198000.0      1
197000.0      1
Name: price, dtype: int64

The top 10 data points are clearly invalid and can either either be removed or fixed based on other data in the table.  Since there are only 10 of 50,000 data points removing them should not significantly impact the data set.

In [16]:
autos = autos.loc[autos['price'] <= 350000, :]
autos['price'].value_counts().sort_index(ascending=False).head(20)

350000.0    1
345000.0    1
299000.0    1
295000.0    1
265000.0    1
259000.0    1
250000.0    1
220000.0    1
198000.0    1
197000.0    1
194000.0    1
190000.0    1
180000.0    1
175000.0    1
169999.0    1
169000.0    1
163991.0    1
163500.0    1
155000.0    1
151990.0    1
Name: price, dtype: int64

#### Remaining Data Points
After cleaning the 'price' column, we are left with 47,645 of 50,000 data points (95.3%).

In [17]:
autos.shape

(47645, 20)

### Price Analysis

In [18]:
autos['price'].describe(percentiles=[.1, .25, .5, .75, .9, .95, .99])

count     47645.000000
mean       6000.707273
std        9110.783444
min         200.000000
10%         650.000000
25%        1300.000000
50%        3190.000000
75%        7500.000000
90%       14500.000000
95%       19999.000000
99%       36000.000000
max      350000.000000
Name: price, dtype: float64

## Odometer
#### Checking for NaN
There are no NaN in 'odometer_km'.

In [19]:
autos['odometer_km'].isnull().sum()

0

### Odometer Readings
Odometer readings range from 5,000km to 150,000km. It is likely due to the nature of the data that the readings are approximations.  It is also possible that anything above 150,000km is not reported and grouped together.

In [20]:
autos['odometer_km'].value_counts().sort_index(ascending=False)

150000.0    30781
125000.0     4998
100000.0     2083
90000.0      1720
80000.0      1407
70000.0      1209
60000.0      1145
50000.0      1007
40000.0       813
30000.0       773
20000.0       747
10000.0       242
5000.0        720
Name: odometer_km, dtype: int64

### Odometer Analysis

In [21]:
autos['odometer_km'].describe()

count     47645.000000
mean     125887.501312
std       39482.911790
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

## Date Analysis
Data stored in ***date_crawled, ad_created, last_seen*** are string types.  ***registration_month, registration_year*** are int64 types.

In [22]:
date_columns = ['date_crawled', 
                'ad_created', 
                'last_seen', 
                'registration_month', 
                'registration_year']

autos[date_columns].head(3)

,date_crawled,ad_created,last_seen,registration_month,registration_year
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54,3,2004
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08,6,1997
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37,7,2009


In [23]:
autos[date_columns].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47645 entries, 0 to 49999
Data columns (total 5 columns):
date_crawled          47645 non-null object
ad_created            47645 non-null object
last_seen             47645 non-null object
registration_month    47645 non-null int64
registration_year     47645 non-null int64
dtypes: int64(2), object(3)
memory usage: 2.2+ MB


### Dates
Stripping the ***date_crawled, ad_created, last_seen*** columns of the time will allow the dates to be easily sorted.  Because the format of the date is a string "YYYY-MM-DD HH:MM:SS" we can take a slice that returns "YYYY-MM-DD" and use this to sort the data.  By appending a new column to the data we avoid losing the time data for future use.

In [24]:
date_crawled_date = autos['date_crawled'].str[0:10]
autos['date_crawled_date'] = date_crawled_date

ad_created_date = autos['ad_created'].str[0:10]
autos['ad_created_date'] = ad_created_date

last_seen_date = autos['last_seen'].str[0:10]
autos['last_seen_date'] = last_seen_date

### Crawl Dates
The data was collected over 34 days, starting 3/5/2016 and ending 4/7/2016.  

In [25]:
print('Days:', autos['date_crawled_date'].unique().shape[0])
print('Start:', autos['date_crawled_date'].min())
print('Finish:', autos['date_crawled_date'].max())

Days: 34
Start: 2016-03-05
Finish: 2016-04-07


On average, data was collected on 1,401 ads per day.  The most and least ads collected in a single day were 1,845 and 65 respectively.

In [26]:
print('Max ads:', autos['date_crawled_date'].value_counts(dropna=False).max())
print('Min ads:', autos['date_crawled_date'].value_counts(dropna=False).min())
print('Avg ads:', autos['date_crawled_date'].value_counts(dropna=False).mean())

Max ads: 1845
Min ads: 65
Avg ads: 1401.3235294117646


In [27]:
print('Number of ads collected:')
print(autos['date_crawled_date'].value_counts(dropna=False, normalize=False).sort_index())
print('Percent of ads collected:')
print(autos['date_crawled_date'].value_counts(dropna=False, normalize=True).sort_index()*100)

Number of ads collected:
2016-03-05    1208
2016-03-06     670
2016-03-07    1715
2016-03-08    1578
2016-03-09    1574
2016-03-10    1540
2016-03-11    1558
2016-03-12    1757
2016-03-13     748
2016-03-14    1742
2016-03-15    1631
2016-03-16    1403
2016-03-17    1503
2016-03-18     611
2016-03-19    1649
2016-03-20    1801
2016-03-21    1780
2016-03-22    1558
2016-03-23    1543
2016-03-24    1397
2016-03-25    1497
2016-03-26    1535
2016-03-27    1487
2016-03-28    1669
2016-03-29    1619
2016-03-30    1615
2016-03-31    1517
2016-04-01    1611
2016-04-02    1700
2016-04-03    1845
2016-04-04    1742
2016-04-05     626
2016-04-06     151
2016-04-07      65
Name: date_crawled_date, dtype: int64
Percent of ads collected:
2016-03-05    2.535418
2016-03-06    1.406234
2016-03-07    3.599538
2016-03-08    3.311995
2016-03-09    3.303600
2016-03-10    3.232238
2016-03-11    3.270018
2016-03-12    3.687690
2016-03-13    1.569944
2016-03-14    3.656207
2016-03-15    3.423234
2016-03-16  

### Ad Dates
Data included ads created on 76 unique days between 6/11/2015 and 4/7/2016.  

In [28]:
print('Days:', autos['ad_created_date'].unique().shape[0])
print('Start:', autos['ad_created_date'].min())
print('Finish:', autos['ad_created_date'].max())

Days: 76
Start: 2015-06-11
Finish: 2016-04-07


In [29]:
print('Max ads:', autos['ad_created_date'].value_counts(dropna=False).max())
print('Min ads:', autos['ad_created_date'].value_counts(dropna=False).min())
print('Avg ads:', autos['ad_created_date'].value_counts(dropna=False).mean())

Max ads: 1857
Min ads: 1
Avg ads: 626.9078947368421


On average, 627 ads were created each day.  

Older dates had few ads and there were large gaps in dates indicating they have been deleted or expired.  For example, there are no ads listed between 6/11/2015 and 8/10/2015.

On the other hand, newer dates showed no gaps and very large number of postings each day.  For example, there were no gaps between 3/9/2016 and 4/7/2016 and most days had well over 1,000 ads created.

In [30]:
print('Number of ads created:')
print(autos['ad_created_date'].value_counts(dropna=False, normalize=False).sort_index())
print('Percent of ads created:')
print(autos['ad_created_date'].value_counts(dropna=False, normalize=True).sort_index()*100)

Number of ads created:
2015-06-11       1
2015-08-10       1
2015-09-09       1
2015-11-10       1
2015-12-05       1
2015-12-30       1
2016-01-03       1
2016-01-07       1
2016-01-10       2
2016-01-13       1
2016-01-14       1
2016-01-16       1
2016-01-22       1
2016-01-27       3
2016-01-29       1
2016-02-01       1
2016-02-02       2
2016-02-05       2
2016-02-07       1
2016-02-08       1
2016-02-09       1
2016-02-11       1
2016-02-12       2
2016-02-14       2
2016-02-16       1
2016-02-17       1
2016-02-18       2
2016-02-19       3
2016-02-20       2
2016-02-21       3
              ... 
2016-03-09    1579
2016-03-10    1526
2016-03-11    1573
2016-03-12    1748
2016-03-13     815
2016-03-14    1676
2016-03-15    1618
2016-03-16    1427
2016-03-17    1487
2016-03-18     641
2016-03-19    1597
2016-03-20    1805
2016-03-21    1790
2016-03-22    1550
2016-03-23    1535
2016-03-24    1394
2016-03-25    1503
2016-03-26    1537
2016-03-27    1484
2016-03-28    1673
2016-03-

To get a more accurate average ads per day, it might be better to only look at a month of data.  The most recent complete month of data was March, 2016.

In [31]:
march_2016_ads = autos.loc[autos['ad_created_date'].str[0:7] =='2016-03', :]
march_2016_ads.head(3)

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,hp,...,fuel_type,brand,unrepaired_damage,ad_created,num_of_pictures,postal_code,last_seen,date_crawled_date,ad_created_date,last_seen_date
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000.0,control,bus,2004,manuell,158,...,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54,2016-03-26,2016-03-26,2016-04-06
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,8990.0,test,limousine,2009,manuell,102,...,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37,2016-03-26,2016-03-26,2016-04-06
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350.0,control,kleinwagen,2007,automatik,71,...,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28,2016-03-12,2016-03-12,2016-03-15


In [32]:
print('Days:', march_2016_ads['ad_created_date'].unique().shape[0])
print('Start:', march_2016_ads['ad_created_date'].min())
print('Finish:', march_2016_ads['ad_created_date'].max())

Days: 31
Start: 2016-03-01
Finish: 2016-03-31


In [33]:
print('Max ads:', march_2016_ads['ad_created_date'].value_counts(dropna=False).max())
print('Min ads:', march_2016_ads['ad_created_date'].value_counts(dropna=False).min())
print('Avg ads:', march_2016_ads['ad_created_date'].value_counts(dropna=False).mean())

Max ads: 1805
Min ads: 5
Avg ads: 1286.3548387096773


On average, 1,286 ads were created each day.  This seems much more accurate.

We also notice that 3/1/2016 through 3/4/2016 had a signficant decrease in ads.  This is approximately 1 month form the crawl date, and might indicate that ads expire approximately 1 month from initial posting.

In [34]:
print('Number of ads created:')
print(march_2016_ads['ad_created_date'].value_counts(dropna=False, normalize=False).sort_index())
print('Percent of ads created:')
print(march_2016_ads['ad_created_date'].value_counts(dropna=False, normalize=True).sort_index()*100)

Number of ads created:
2016-03-01       5
2016-03-02       5
2016-03-03      40
2016-03-04      71
2016-03-05    1093
2016-03-06     730
2016-03-07    1654
2016-03-08    1578
2016-03-09    1579
2016-03-10    1526
2016-03-11    1573
2016-03-12    1748
2016-03-13     815
2016-03-14    1676
2016-03-15    1618
2016-03-16    1427
2016-03-17    1487
2016-03-18     641
2016-03-19    1597
2016-03-20    1805
2016-03-21    1790
2016-03-22    1550
2016-03-23    1535
2016-03-24    1394
2016-03-25    1503
2016-03-26    1537
2016-03-27    1484
2016-03-28    1673
2016-03-29    1618
2016-03-30    1606
2016-03-31    1519
Name: ad_created_date, dtype: int64
Percent of ads created:
2016-03-01    0.012539
2016-03-02    0.012539
2016-03-03    0.100308
2016-03-04    0.178047
2016-03-05    2.740928
2016-03-06    1.830629
2016-03-07    4.147754
2016-03-08    3.957168
2016-03-09    3.959676
2016-03-10    3.826767
2016-03-11    3.944630
2016-03-12    4.383479
2016-03-13    2.043785
2016-03-14    4.202924
2016-0

### Registration Year
Vehicle registration year has data ranging from 1000 to 9999.

In [35]:
autos['registration_year'].describe()

count    47645.000000
mean      2004.800084
std         88.423872
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

In [36]:
autos['registration_year'].value_counts().sort_index().head(5)

1000    1
1001    1
1111    1
1800    2
1910    2
Name: registration_year, dtype: int64

In [37]:
autos['registration_year'].value_counts().sort_index().tail(12)

2018    468
2019      1
2800      1
4100      1
4500      1
4800      1
5000      3
5911      1
6200      1
8888      1
9000      1
9999      3
Name: registration_year, dtype: int64

Since the data was collected in 2016, and the auto industry tends to release vehicles in the previous calendar year, vehicles will be considered valid with a year less than or equal to 2017 resulting in a loss of 488 data points.

Classic vehicles are commonly sold on eBay, so it is possible for older vehicle years to be valid.  All dates after 1900 will be considered valid resulting in a loss of 5 data points.

In [38]:
print('Size before removal:', autos.shape[0])
autos = autos.loc[autos['registration_year'] >= 1900, :]
autos = autos.loc[autos['registration_year'] <= 2017, :]
print('Size after removal:', autos.shape[0])

Size before removal: 47645
Size after removal: 47157


In [39]:
print('Max ads by registration year:', autos['registration_year'].value_counts(dropna=False).max())
print('Min ads by registration year:', autos['registration_year'].value_counts(dropna=False).min())
print('Avg ads by registration year:', autos['registration_year'].value_counts(dropna=False).mean())

Max ads by registration year: 3019
Min ads by registration year: 1
Avg ads by registration year: 596.9240506329114


#### Results
Cars listed on eBay were relatively old vehicles (5 to 10 years old) and the classic or collectors car market is likely to be a very small portion of listings.

From the results below we can see the following:

1) 10% of cars were 2012 or newer (about 5 years old at the time)
2) 50% of cars listed were 2004 or older (over 10 years old)
3) 1% of cars were 1979 or older indicating a small classic car market

In [40]:
autos['registration_year'].describe(percentiles=[.01, .05, .50, .75, .90, .95])

count    47157.000000
mean      2003.400322
std          7.393552
min       1910.000000
1%        1979.000000
5%        1993.000000
50%       2004.000000
75%       2008.000000
90%       2012.000000
95%       2016.000000
max       2017.000000
Name: registration_year, dtype: float64

### Registration Month
registration_month ranges from 0 to 12.  Valid values should be 1 (January) to 12 (December).

In [41]:
autos['registration_month'].describe()

count    47157.000000
mean         5.834531
std          3.662464
min          0.000000
25%          3.000000
50%          6.000000
75%          9.000000
max         12.000000
Name: registration_month, dtype: float64

There are about 4,000 data points with 0 registration_month.  Since we don't know the month of registration, we will set it to "Unknown".

In [42]:
autos['registration_month'].value_counts()

3     4897
6     4177
0     4000
5     3953
4     3941
7     3785
10    3528
12    3297
9     3258
11    3255
1     3125
8     3064
2     2877
Name: registration_month, dtype: int64

In [43]:
autos.loc[autos['registration_month']==0,'registration_month'] = "Unknown"
autos['registration_month'].value_counts()

3          4897
6          4177
Unknown    4000
5          3953
4          3941
7          3785
10         3528
12         3297
9          3258
11         3255
1          3125
8          3064
2          2877
Name: registration_month, dtype: int64

## Brand Analysis
There are 40 unique brands represented in the data.

In [44]:
brands = autos['brand'].unique()
print('Unique brands:', len(brands))

Unique brands: 40


The top 10 brands are shown below.  The top 5 brands, all German companies, represent a very large portion of the market.  We will focus on these 5 brands for the detailed anlaysis.

In [45]:
(autos['brand'].value_counts(normalize=True) * 100).head(10)

volkswagen       21.258774
bmw              11.001548
opel             10.670738
mercedes_benz     9.703756
audi              8.709205
ford              6.864304
renault           4.711920
peugeot           2.968806
fiat              2.531968
seat              1.853383
Name: brand, dtype: float64

Volkswagen, BMW, Opel, Mercedes Benz and Audi have a combined market share of over 61%.


In [46]:
(autos['brand'].value_counts(normalize=True) * 100).head(5).sum()

61.34402103611341

#### Average Price

In [47]:
avg_price = []
avg_price_dic = {}
for brand in brands:
    avg_price.append((round(autos.loc[autos['brand']==brand, 'price'].mean(), 2), brand))
    avg_price_dic[brand] = round(autos.loc[autos['brand']==brand, 'price'].mean(), 2)
avg_price.sort()

print("Top 5 Expensive Brands:")
for i in range(-1,-5,-1):
    print(avg_price[i][1].title(), '${:,}'.format(avg_price[i][0]))

print("\nTop 5 Cheapest Brands:")
for i in range(0,5):
    print(avg_price[i][1].title(), '${:,}'.format(avg_price[i][0]))



Top 5 Expensive Brands:
Porsche $46,788.44
Land_Rover $18,934.27
Sonstige_Autos $12,868.23
Jaguar $12,129.6

Top 5 Cheapest Brands:
Daewoo $1,119.45
Rover $1,609.72
Daihatsu $1,650.58
Trabant $1,953.3
Renault $2,516.23


#### Average Milage

In [48]:
avg_milage = []
avg_milage_dic = {}
for brand in brands:
    avg_milage.append((round(autos.loc[autos['brand']==brand, 'odometer_km'].mean(), 2), brand))
    avg_milage_dic[brand] = round(autos.loc[autos['brand']==brand, 'odometer_km'].mean(), 2)
avg_milage.sort()

print("Top 5 Highest Milage Brands:")
for i in range(-1,-5,-1):
    print(avg_milage[i][1].title(), '{:,}km'.format(avg_milage[i][0]))

print("\nTop 5 Lowest Milage Brands:")
for i in range(0,5):
    print(avg_milage[i][1].title(), '{:,}km'.format(avg_milage[i][0]))

Top 5 Highest Milage Brands:
Saab 144,415.58km
Volvo 138,735.5km
Rover 138,046.88km
Chrysler 133,125.0km

Top 5 Lowest Milage Brands:
Trabant 55,245.9km
Lada 83,518.52km
Dacia 84,218.75km
Mini 89,302.88km
Sonstige_Autos 90,995.48km


#### Value Index
Below we create a value index to attempt to capture brand value.  As the average sales price increases, the brand value proportionally increases.  As the milage increases, similarly the brand value increases.  This is because a brand that can sell at higher average price with higher miles is looked at as having more value.

In [49]:
avg_price_series = pd.Series(avg_price_dic)
avg_milage_series = pd.Series(avg_milage_dic)

price_milage = pd.DataFrame(avg_price_series, columns=['mean_price'])
price_milage['odometer_km'] = avg_milage_series
price_milage['value_index'] = avg_price_series * avg_milage_series / 10000000

price_milage

,mean_price,odometer_km,value_index
alfa_romeo,4104.55,131603.17,54.017179
audi,9310.89,129534.94,120.608558
bmw,8352.23,132883.58,110.987422
chevrolet,6787.42,100355.81,68.115703
chrysler,3539.92,133125.00,47.125185
citroen,3796.90,120141.79,45.616636
dacia,5920.38,84218.75,49.860700
daewoo,1119.45,120479.45,13.487072
daihatsu,1650.58,116196.58,19.179175
fiat,2898.15,117211.06,33.969523


#### Top 5 Value Index Cars

In [50]:
price_milage.sort_values(by='value_index', ascending=False).head(5)

,mean_price,odometer_km,value_index
porsche,46788.44,98476.70,460.757117
land_rover,18934.27,118333.33,224.055522
jaguar,12129.60,125428.57,152.139838
jeep,11590.21,127102.80,147.314814
audi,9310.89,129534.94,120.608558


#### Bottom 5 Value Index Cars

In [51]:
price_milage.sort_values(by='value_index', ascending=True).head(5)

,mean_price,odometer_km,value_index
trabant,1953.30,55245.90,10.791182
daewoo,1119.45,120479.45,13.487072
daihatsu,1650.58,116196.58,19.179175
rover,1609.72,138046.88,22.221682
lada,2688.30,83518.52,22.452284
